# BearCart E-Commerce Dataset – Data Cleaning & Consistency Processing

This notebook documents the complete data cleaning and consistency validation process performed on the BearCart e-commerce dataset.  
The cleaned datasets produced here are used for dashboarding and business analysis.


## 1. Library Imports and Environment Setup

In [ ]:
import os

os.makedirs("Cleaned_Dataset", exist_ok=True)
os.makedirs("Source_Code", exist_ok=True)
os.makedirs("Documentation", exist_ok=True)


In [ ]:
import zipfile

zip_path = "/content/BearCart Dataset-20251218T042526Z-1-001.zip"
extract_path = "/content/Raw_Data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


## 1. Library Imports and Setup

In [ ]:
import pandas as pd
import numpy as np


## 2. Loading Raw Data


In [ ]:
base = "/content/Raw_Data/BearCart Dataset"

products = pd.read_csv(f"{base}/products.csv")
orders = pd.read_csv(f"{base}/orders.csv")
order_items = pd.read_csv(f"{base}/order_items.csv")
refunds = pd.read_csv(f"{base}/order_item_refunds.csv")
sessions = pd.read_csv(f"{base}/website_sessions.csv")
pageviews = pd.read_csv(f"{base}/website_pageviews.csv")


In [ ]:
for name, df in {
    "products": products,
    "orders": orders,
    "order_items": order_items,
    "refunds": refunds,
    "sessions": sessions,
    "pageviews": pageviews
}.items():
    print(f"\n{name.upper()} COLUMNS")
    print(list(df.columns))



PRODUCTS COLUMNS
['product_id', 'created_at', 'product_name']

ORDERS COLUMNS
['order_id', 'created_at', 'website_session_id', 'user_id', 'primary_product_id', 'items_purchased', 'price_usd', 'cogs_usd']

ORDER_ITEMS COLUMNS
['order_item_id', 'created_at', 'order_id', 'product_id', 'is_primary_item', 'price_usd', 'cogs_usd']

REFUNDS COLUMNS
['order_item_refund_id', 'created_at', 'order_item_id', 'order_id', 'refund_amount_usd']

SESSIONS COLUMNS
['website_session_id', 'created_at', 'user_id', 'is_repeat_session', 'utm_source', 'utm_campaign', 'utm_content', 'device_type', 'http_referer']

PAGEVIEWS COLUMNS
['website_pageview_id', 'created_at', 'website_session_id', 'pageview_url']


## 3. Initial Data Inspection


In [ ]:
# Check shape and columns of each dataset
datasets = {
    "products": products,
    "orders": orders,
    "order_items": order_items,
    "refunds": refunds,
    "sessions": sessions,
    "pageviews": pageviews
}

for name, df in datasets.items():
    print(f"\n{name.upper()}")
    print("Shape:", df.shape)
    print(df.head())



PRODUCTS
Shape: (4, 3)
   product_id           created_at                 product_name
0           1  2012-03-19 08:00:00       The Original Mr. Fuzzy
1           2  2013-01-06 13:00:00        The Forever Love Bear
2           3  2013-12-12 09:00:00  The Birthday Sugar Panda-v2
3           4  2014-02-05 10:00:00   The Hudson River Mini bear

ORDERS
Shape: (32313, 8)
   order_id           created_at  website_session_id  user_id  \
0         1  2012-03-19 10:42:46                  20     20.0   
1         2  2012-03-19 19:27:37                 104    104.0   
2         3  2012-03-20 06:44:45                 147    147.0   
3         4  2012-03-20 09:41:45                 160    160.0   
4         5  2012-03-20 11:28:15                 177    177.0   

   primary_product_id  items_purchased  price_usd  cogs_usd  
0                   1                1      49.99     19.49  
1                   1                1      49.99     19.49  
2                   1                1      49.99    

## Phase 1: Structural Data Cleaning

This phase focuses on essential and non-controversial cleaning steps to ensure structural consistency across all datasets.  
These steps do not alter the meaning of the data and are required before any analysis or inconsistency detection.


### 1.1 Column Name Standardization

Column names are standardized across all datasets by converting them to lowercase, removing extra spaces, and replacing spaces with underscores.  
This ensures consistency and prevents errors during joins, aggregations, and dashboard development.


In [ ]:
def standardize_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
    )
    return df

products = standardize_columns(products)
orders = standardize_columns(orders)
order_items = standardize_columns(order_items)
refunds = standardize_columns(refunds)
sessions = standardize_columns(sessions)
pageviews = standardize_columns(pageviews)


### Verification: Column Names After Standardization

The following output verifies that all column names now follow a consistent naming convention.


In [ ]:
for name, df in {
    "products": products,
    "orders": orders,
    "order_items": order_items,
    "refunds": refunds,
    "sessions": sessions,
    "pageviews": pageviews
}.items():
    print(f"\n{name.upper()} COLUMNS")
    print(list(df.columns))



PRODUCTS COLUMNS
['product_id', 'created_at', 'product_name']

ORDERS COLUMNS
['order_id', 'created_at', 'website_session_id', 'user_id', 'primary_product_id', 'items_purchased', 'price_usd', 'cogs_usd']

ORDER_ITEMS COLUMNS
['order_item_id', 'created_at', 'order_id', 'product_id', 'is_primary_item', 'price_usd', 'cogs_usd']

REFUNDS COLUMNS
['order_item_refund_id', 'created_at', 'order_item_id', 'order_id', 'refund_amount_usd']

SESSIONS COLUMNS
['website_session_id', 'created_at', 'user_id', 'is_repeat_session', 'utm_source', 'utm_campaign', 'utm_content', 'device_type', 'http_referer']

PAGEVIEWS COLUMNS
['website_pageview_id', 'created_at', 'website_session_id', 'pageview_url']


### 1.2 Data Type Correction

Data types are corrected to ensure logical consistency.  
Identifier columns are converted to integer types, and timestamp columns are converted to datetime format.


In [ ]:
for df in [products, orders, order_items, refunds, sessions, pageviews]:
    df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce")


Identifier columns should represent discrete entities and must not be stored as floating-point values.  
These columns are converted to integer types where applicable.


In [ ]:
id_columns = [
    "product_id",
    "order_id",
    "order_item_id",
    "order_item_refund_id",
    "website_session_id",
    "user_id"
]

for df in [products, orders, order_items, refunds, sessions, pageviews]:
    for col in id_columns:
        if col in df.columns:
            df[col] = df[col].astype("Int64")


### Verification: Data Types After Correction

The following output confirms that identifier columns are stored as integers and timestamp columns are stored in datetime format.


In [ ]:
for name, df in {
    "products": products,
    "orders": orders,
    "order_items": order_items,
    "refunds": refunds,
    "sessions": sessions,
    "pageviews": pageviews
}.items():
    print(f"\n{name.upper()} DATA TYPES")
    print(df.dtypes)



PRODUCTS DATA TYPES
product_id               Int64
created_at      datetime64[ns]
product_name            object
dtype: object

ORDERS DATA TYPES
order_id                       Int64
created_at            datetime64[ns]
website_session_id             Int64
user_id                        Int64
primary_product_id             int64
items_purchased                int64
price_usd                    float64
cogs_usd                     float64
dtype: object

ORDER_ITEMS DATA TYPES
order_item_id               Int64
created_at         datetime64[ns]
order_id                    Int64
product_id                  Int64
is_primary_item             int64
price_usd                 float64
cogs_usd                  float64
dtype: object

REFUNDS DATA TYPES
order_item_refund_id             Int64
created_at              datetime64[ns]
order_item_id                    Int64
order_id                         Int64
refund_amount_usd              float64
dtype: object

SESSIONS DATA TYPES
website_session_i

### Phase 1 Summary

Structural data cleaning was performed to establish a consistent and reliable foundation across all datasets.
Column names were standardized to a uniform naming convention, and data types were corrected by converting identifier fields to integers and timestamp fields to datetime format.
These changes ensured schema consistency without altering any business meaning or data values.


## Phase 2: Row-Level Data Cleaning

This phase focuses on cleaning issues that occur at the row level, such as duplicate records and missing values.
All actions are performed conservatively to avoid unintended data loss or distortion of business logic.


### 2.1 Duplicate Record Identification and Removal

Duplicate rows can inflate metrics such as revenue, order counts, and sessions.
Only exact duplicate records are removed in this step.


In [ ]:
datasets = {
    "products": products,
    "orders": orders,
    "order_items": order_items,
    "refunds": refunds,
    "sessions": sessions,
    "pageviews": pageviews
}

for name, df in datasets.items():
    before = df.shape[0]
    df.drop_duplicates(inplace=True)
    after = df.shape[0]
    print(f"{name}: {before - after} duplicate rows removed")


products: 0 duplicate rows removed
orders: 0 duplicate rows removed
order_items: 0 duplicate rows removed
refunds: 0 duplicate rows removed
sessions: 9457 duplicate rows removed
pageviews: 0 duplicate rows removed


The output above confirms the number of duplicate rows removed from each dataset.
sessions table had 9457 duplicate rows so they are removed.


### 2.2 Missing Value Assessment

Missing values are examined across all datasets.
Only rows missing critical identifier fields are considered for removal.
Non-critical missing values are preserved to maintain data authenticity.


In [ ]:
for name, df in datasets.items():
    print(f"\n{name.upper()} – Missing Values")
    print(df.isnull().sum())



PRODUCTS – Missing Values
product_id      0
created_at      0
product_name    0
dtype: int64

ORDERS – Missing Values
order_id                 0
created_at               0
website_session_id       0
user_id               1616
primary_product_id       0
items_purchased          0
price_usd                0
cogs_usd                 0
dtype: int64

ORDER_ITEMS – Missing Values
order_item_id      0
created_at         0
order_id           0
product_id         0
is_primary_item    0
price_usd          0
cogs_usd           0
dtype: int64

REFUNDS – Missing Values
order_item_refund_id    0
created_at              0
order_item_id           0
order_id                0
refund_amount_usd       0
dtype: int64

SESSIONS – Missing Values
website_session_id         0
created_at                 0
user_id                    0
is_repeat_session          0
utm_source             74056
utm_campaign          129994
utm_content            83328
device_type                0
http_referer           39917
dtype

#### Missing Value Handling Rules

- Rows missing primary identifier columns are removed.
- Rows missing non-critical or optional fields are retained.
- No artificial values are inserted to replace missing data.


In [ ]:
# Drop rows where primary identifiers are missing
products = products.dropna(subset=["product_id"])
orders = orders.dropna(subset=["order_id"])
order_items = order_items.dropna(subset=["order_item_id"])
refunds = refunds.dropna(subset=["order_item_refund_id"])
sessions = sessions.dropna(subset=["website_session_id"])
pageviews = pageviews.dropna(subset=["website_pageview_id"])


### Verification: Missing Values After Cleaning

The following output confirms that critical identifier fields no longer contain missing values.


In [ ]:
for name, df in datasets.items():
    print(f"\n{name.upper()} – Missing Critical IDs")
    print(df.isnull().sum())



PRODUCTS – Missing Critical IDs
product_id      0
created_at      0
product_name    0
dtype: int64

ORDERS – Missing Critical IDs
order_id                 0
created_at               0
website_session_id       0
user_id               1616
primary_product_id       0
items_purchased          0
price_usd                0
cogs_usd                 0
dtype: int64

ORDER_ITEMS – Missing Critical IDs
order_item_id      0
created_at         0
order_id           0
product_id         0
is_primary_item    0
price_usd          0
cogs_usd           0
dtype: int64

REFUNDS – Missing Critical IDs
order_item_refund_id    0
created_at              0
order_item_id           0
order_id                0
refund_amount_usd       0
dtype: int64

SESSIONS – Missing Critical IDs
website_session_id         0
created_at                 0
user_id                    0
is_repeat_session          0
utm_source             74056
utm_campaign          129994
utm_content            83328
device_type                0
http

### Phase 2 Summary

No records with missing primary identifiers were found across the datasets.
Missing values observed in user identifiers and marketing attribution fields were retained, as they represent valid real-world scenarios such as guest users and direct traffic.


## Phase 3: Referential Integrity and Cross-Table Consistency

This phase validates the logical relationships between datasets to ensure referential integrity.
All inconsistencies are identified and documented without altering the original records.


### 3.1 Orders Referencing Valid Website Sessions

Each order should be associated with a valid website session.
This check ensures that all orders reference existing sessions.


In [ ]:
invalid_orders = orders[
    ~orders["website_session_id"].isin(sessions["website_session_id"])
]

print("Orders with invalid website_session_id:", len(invalid_orders))


Orders with invalid website_session_id: 0


### 3.2 Pageviews Referencing Valid Website Sessions

Each pageview must be linked to a valid website session.
This ensures accurate session-level navigation analysis.


In [ ]:
invalid_pageviews = pageviews[
    ~pageviews["website_session_id"].isin(sessions["website_session_id"])
]

print("Pageviews with invalid website_session_id:", len(invalid_pageviews))


Pageviews with invalid website_session_id: 0


### 3.3 Order Items Referencing Valid Orders

Each order item should reference a valid order.
This check confirms that no orphaned order items exist.


In [ ]:
invalid_order_items = order_items[
    ~order_items["order_id"].isin(orders["order_id"])
]

print("Order items with invalid order_id:", len(invalid_order_items))


Order items with invalid order_id: 0


### 3.4 Refunds Referencing Valid Order Items

Refund records should reference valid order items.
This ensures refund data can be accurately tied to purchased products.


In [ ]:
invalid_refunds = refunds[
    ~refunds["order_item_id"].isin(order_items["order_item_id"])
]

print("Refunds with invalid order_item_id:", len(invalid_refunds))


Refunds with invalid order_item_id: 0


### 3.5 Refunds Referencing Valid Orders

Refund records are also checked against orders to ensure consistency across tables.


In [ ]:
invalid_refund_orders = refunds[
    ~refunds["order_id"].isin(orders["order_id"])
]

print("Refunds with invalid order_id:", len(invalid_refund_orders))


Refunds with invalid order_id: 0


### Phase 3 Summary

All referential integrity checks returned zero invalid records.
This confirms that relationships across sessions, orders, order items, and refunds are fully consistent.



## Phase 4: Business Logic Consistency Checks

This phase evaluates whether the data aligns with expected business rules such as item counts, pricing logic, and refund behavior.
Any inconsistencies identified are documented without modifying the original records.


### 4.1 Items Purchased vs Order Items Count

The number of items purchased recorded in the orders table should match the actual count of order items associated with each order.
This check validates consistency between order-level and item-level data.


In [ ]:
# Count actual items per order from order_items
item_counts = (
    order_items
    .groupby("order_id")
    .size()
    .reset_index(name="actual_item_count")
)

# Merge with orders table
item_check = orders.merge(
    item_counts,
    on="order_id",
    how="left"
)

# Identify mismatches
item_check["item_count_mismatch"] = (
    item_check["items_purchased"] != item_check["actual_item_count"]
)

item_check["item_count_mismatch"].value_counts()


,count
item_count_mismatch,
False,32313


The item count recorded at the order level exactly matches the number of order items for all orders.
No inconsistencies were identified between order-level and item-level item counts.


### 4.2 Order Price vs Sum of Order Item Prices

The total order price should match the sum of individual item prices for that order.
This check validates revenue consistency across tables.


In [ ]:
# Sum item prices per order
order_item_prices = (
    order_items
    .groupby("order_id")["price_usd"]
    .sum()
    .reset_index(name="calculated_order_price")
)

# Merge with orders
price_check = orders.merge(
    order_item_prices,
    on="order_id",
    how="left"
)

# Compare prices
price_check["price_mismatch"] = (
    price_check["price_usd"].round(2) !=
    price_check["calculated_order_price"].round(2)
)

price_check["price_mismatch"].value_counts()


,count
price_mismatch,
False,29728
True,2585


Out of 32,313 orders, 2,585 orders show a mismatch between the order-level price and the sum of item-level prices.
This behavior is likely due to discounts, bundled pricing, or order-level adjustments.
Both values are retained, with order-level pricing recommended for revenue calculations.


In [ ]:
orders["price_mismatch_flag"] = orders["order_id"].isin(
    price_check.loc[price_check["price_mismatch"], "order_id"]
)


A boolean flag was added to the orders table to identify orders where the order-level price differs from the sum of item-level prices.
This enables transparent analysis of discounted or adjusted orders without altering original financial data.


In [ ]:
orders["price_mismatch_flag"].value_counts()


,count
price_mismatch_flag,
False,29728
True,2585


### 4.3 Refund Amount vs Item Price

Refund amounts should not exceed the price of the associated order item.
This check validates the financial correctness of refund records.


In [ ]:
# Merge refunds with item prices
refund_check = refunds.merge(
    order_items[["order_item_id", "price_usd"]],
    on="order_item_id",
    how="left"
)

# Identify invalid refunds
refund_check["invalid_refund"] = (
    refund_check["refund_amount_usd"] >
    refund_check["price_usd"]
)

refund_check["invalid_refund"].value_counts()


,count
invalid_refund,
False,1731


All refund records were validated against item prices.
No cases were found where the refund amount exceeded the corresponding item price, confirming financial correctness of refund data.


### Phase 4 Summary

Business logic consistency checks were conducted to validate item counts, pricing logic, and refund behavior across the dataset.
Order-level item counts were found to be fully consistent with item-level records, and all refund amounts were within valid item price limits.
Order-level pricing differences identified for certain orders were transparently documented and flagged to account for discounts or adjustments, without modifying original financial data.


## Phase 5: Soft Standardization for Analysis and Dashboarding

This phase focuses on light standardization of categorical and text fields to improve grouping, filtering, and visualization.
These changes do not alter business meaning or numerical values.


### 5.1 Standardizing Categorical Text Fields

Categorical fields such as device type and marketing attributes are standardized by trimming whitespace and converting text to lowercase.
This prevents duplicate categories caused by formatting differences.


In [ ]:
# Columns to standardize (only text-based categorical fields)
categorical_columns = {
    "sessions": ["device_type", "utm_source", "utm_campaign", "utm_content", "http_referer"],
    "products": ["product_name"],
    "pageviews": ["pageview_url"]
}

# Apply standardization
for df_name, cols in categorical_columns.items():
    df = globals()[df_name]
    for col in cols:
        df[col] = (
            df[col]
            .astype("string")
            .str.strip()
            .str.lower()
        )


Missing values in marketing attribution fields are intentionally preserved.
These represent valid scenarios such as direct traffic or unattributed sessions and should not be artificially filled.


### 5.2 Binary Field Validation

Binary indicator fields are validated to ensure they contain only expected values.


In [ ]:
binary_checks = {
    "sessions.is_repeat_session": sessions["is_repeat_session"].unique(),
    "order_items.is_primary_item": order_items["is_primary_item"].unique(),
    "orders.price_mismatch_flag": orders["price_mismatch_flag"].unique()
}

for name, values in binary_checks.items():
    print(name, "->", values)


sessions.is_repeat_session -> [0 1]
order_items.is_primary_item -> [1 0]
orders.price_mismatch_flag -> <BooleanArray>
[False, True]
Length: 2, dtype: boolean


Binary indicator fields were validated and confirmed to contain only expected values.
No invalid or ambiguous states were found.


### 5.3 URL and Path Consistency

Page URLs are standardized to ensure consistent path-based analysis in funnel and navigation reporting.


In [ ]:
pageviews["pageview_url"].value_counts().head(10)


,count
pageview_url,
/products,261231
/the-original-mr-fuzzy,162525
/home,137576
/lander-2,131170
/cart,94953
/lander-3,79000
/lander-5,68166
/shipping,64484
/billing-2,48441


Pageview URLs were reviewed and confirmed to be consistently formatted.
No case-sensitive or whitespace-based duplicates were found, ensuring reliable page-level and funnel analysis.


### Phase 5 Summary

Soft standardization was applied to text-based categorical fields to ensure consistent grouping and filtering during analysis and dashboarding.
Categorical values were trimmed and converted to lowercase, binary indicator fields were validated, and URL paths were confirmed to be consistently formatted.
Missing values in marketing and attribution fields were intentionally preserved to reflect real-world user behavior.
No numerical values or business logic were altered in this phase.


## Phase 6: Final Cleaned Dataset Export and Submission Preparation

This phase finalizes the cleaned datasets after all structural, row-level, referential, business logic, and standardization checks.
The outputs generated in this phase are submission-ready and intended for dashboard development and business analysis.


### 6.1 Creation of Final Cleaned Tables

Final cleaned copies of each dataset are created to preserve the processed state of the data.
Raw datasets are not overwritten to maintain traceability.


In [ ]:
products_cleaned = products.copy()
orders_cleaned = orders.copy()
order_items_cleaned = order_items.copy()
refunds_cleaned = refunds.copy()
sessions_cleaned = sessions.copy()
pageviews_cleaned = pageviews.copy()


### 6.2 Exporting Cleaned Datasets

The cleaned datasets are exported as CSV files using clear and consistent naming conventions.
These files represent the final output of the data cleaning and consistency handling process.


In [ ]:
products_cleaned.to_csv("Cleaned_Dataset/cleaned_products.csv", index=False)
orders_cleaned.to_csv("Cleaned_Dataset/cleaned_orders.csv", index=False)
order_items_cleaned.to_csv("Cleaned_Dataset/cleaned_order_items.csv", index=False)
refunds_cleaned.to_csv("Cleaned_Dataset/cleaned_refunds.csv", index=False)
sessions_cleaned.to_csv("Cleaned_Dataset/cleaned_sessions.csv", index=False)
pageviews_cleaned.to_csv("Cleaned_Dataset/cleaned_pageviews.csv", index=False)


### Verification of Exported Files

All cleaned datasets were successfully exported and verified for completeness.
These files are ready for dashboard integration and submission.


### Overall Data Cleaning Summary

The BearCart e-commerce dataset was cleaned and processed using a structured, multi-phase approach.
Structural consistency, row-level quality, referential integrity, business logic validation, and soft standardization were applied.
Identified inconsistencies were transparently documented rather than forcibly corrected to preserve data authenticity.
The resulting datasets are reliable, consistent, and suitable for analysis and dashboarding.


In [ ]:
!zip -r BearCart_Cleaned_Dataset.zip Cleaned_Dataset


  adding: Cleaned_Dataset/ (stored 0%)
  adding: Cleaned_Dataset/cleaned_refunds.csv (deflated 67%)
  adding: Cleaned_Dataset/cleaned_pageviews.csv (deflated 80%)
  adding: Cleaned_Dataset/cleaned_sessions.csv (deflated 87%)
  adding: Cleaned_Dataset/cleaned_products.csv (deflated 32%)
  adding: Cleaned_Dataset/cleaned_order_items.csv (deflated 81%)
  adding: Cleaned_Dataset/cleaned_orders.csv (deflated 77%)
